# FAA Aircraft Registration Database

In [1]:
import pandas as pd
import requests
import matplotlib as mpl
import geopandas as gpd
import descartes
import geojson
import json
import numpy as np
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_colwidth', 100)

In [2]:
# https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/

In [3]:
master = pd.read_csv('/Users/mhustiles/data/data/FAA_Aircraft_Database/MASTER.txt', low_memory=False)

In [4]:
reference = pd.read_csv('/Users/mhustiles/data/data/FAA_Aircraft_Database/ACFTREF.txt', low_memory=False)

In [5]:
master.columns = master.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [6]:
reference.columns = reference.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [7]:
reference.loc[(reference['type_acft'] == '4'),'type']='Fixed wing single engine'
reference.loc[(reference['type_acft'] == '5'),'type']='Fixed wing multi engine'
reference.loc[(reference['type_acft'] == '6'),'type']='Rotorcraft'

In [8]:
reference.head()

,code,mfr,model,type_acft,type_eng,ac_cat,build_cert_ind,no_eng,no_seats,ac_weight,speed,unnamed:_11,type
0,0020901,AAR AIRLIFT GROUP INC,UH-60A,6,3,1,0,2,15,CLASS 3,0,nan,Rotorcraft
1,0030109,EXLINE ACE-C,ACE-C,4,1,1,1,1,1,CLASS 1,82,nan,Fixed wing single engine
2,003010D,DELEBAUGH,P,4,1,1,1,1,1,CLASS 1,82,nan,Fixed wing single engine
3,003010H,DAL PORTO,BABY ACE D,4,1,1,1,1,1,CLASS 1,82,nan,Fixed wing single engine
4,003010P,DUNN,BABY ACE,4,1,1,1,1,1,CLASS 1,82,nan,Fixed wing single engine


In [9]:
master.name = master.name.str.strip()
master.certification = master.certification.str.strip()
master.type_registrant = master.type_registrant.str.strip()

In [10]:
owners = master.groupby(['name', 'type_registrant', 'certification']).agg('size').reset_index(name='planes')

In [11]:
owners.head()

,name,type_registrant,certification,planes
0,,,,160
1,,,1,33
2,,,1B,2
3,,,1N,37
4,,,1NA,2


In [12]:
corptrans = owners[(owners['planes'] > 20) & (owners['certification'] == '1T')]

In [13]:
airlines = ['DELTA AIR LINES INC', 'AMERICAN AIRLINES INC', 'SKYWEST AIRLINES INC', 'SWIFT AIR LLC',\
'UNITED AIRLINES INC', 'SOUTHWEST AIRLINES CO', 'JETBLUE AIRWAYS CORP', 'HORIZON AIR INDUSTRIES',\
'ALASKA AIRLINES INC', 'REPUBLIC AIRWAYS INC', 'SPIRIT AIRLINES INC', 'MESA AIRLINES INC']

In [14]:
airlines

['DELTA AIR LINES INC',
 'AMERICAN AIRLINES INC',
 'SKYWEST AIRLINES INC',
 'SWIFT AIR LLC',
 'UNITED AIRLINES INC',
 'SOUTHWEST AIRLINES CO',
 'JETBLUE AIRWAYS CORP',
 'HORIZON AIR INDUSTRIES',
 'ALASKA AIRLINES INC',
 'REPUBLIC AIRWAYS INC',
 'SPIRIT AIRLINES INC',
 'MESA AIRLINES INC']

In [15]:
cargo = ['UNITED PARCEL SERVICE CO','FEDERAL EXPRESS CORP', 'FEDERAL EXPRESS CORPORATION',\
            'CARGO AIRCRAFT MANAGEMENT INC', 'ATLAS AIR INC']

In [16]:
cargo

['UNITED PARCEL SERVICE CO',
 'FEDERAL EXPRESS CORP',
 'FEDERAL EXPRESS CORPORATION',
 'CARGO AIRCRAFT MANAGEMENT INC',
 'ATLAS AIR INC']

In [17]:
private = ['NETJETS SALES INC', 'FLEXJET LLC']

In [18]:
corptrans.sort_values(by='planes', ascending=False).head()

,name,type_registrant,certification,planes
220089,WELLS FARGO TRUST CO NA TRUSTEE,3,1T,1231
54055,DELTA AIR LINES INC,3,1T,979
8943,AMERICAN AIRLINES INC,3,1T,910
211582,UNITED AIRLINES INC,3,1T,861
16036,BANK OF UTAH TRUSTEE,3,1T,787


In [19]:
airline_planes = master

In [20]:
airline_planes_slim = airline_planes[['name','n_number']]

In [21]:
airline_planes_slim.to_csv('output/airline_planes.csv', index=False)

### Airline planes

In [22]:
american = master[master['name'].str.contains('AMERICAN AIRLINES')]

In [23]:
american.iloc[583]

n_number                                                         766AN
serial_number                           32880                         
mfr_mdl_code                                                   1385213
eng_mfr_mdl                                                      54555
year_mfr                                                          2003
type_registrant                                                      3
name                                             AMERICAN AIRLINES INC
street                               1 SKYVIEW DR                     
street2                              MD 8B369                         
city                                                FORT WORTH        
state                                                               TX
zip_code                                                    76155     
region                                                               2
county                                                             439
countr

In [24]:
master['n_number'] = 'N' + master['n_number']

In [25]:
src = master.merge(reference, left_on='mfr_mdl_code', right_on='code')

In [26]:
src[src['n_number'] == 'N302FD']

,n_number,serial_number,mfr_mdl_code,eng_mfr_mdl,year_mfr,type_registrant,name,street,street2,city,...,type_acft,type_eng,ac_cat,build_cert_ind,no_eng,no_seats,ac_weight,speed,unnamed:_11,type
207667,N302FD,41239,0260106,52174,2011,5,LOS ANGELES CITY,C/O LOS ANGELES FIRE DEPARTMENT,AIR OPS SECTION 16617 ARMINTA ST,VAN NUYS,...,6,3,1,0,2,17,CLASS 3,0,nan,Rotorcraft


In [27]:
lapd = ['AIR SUPPORT DIVISION', 
        'LAPD', 
        'LOS ANGELES POLICE', 'LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION']

In [28]:
lapd_county = ['LOS ANGELES COUNTY',
               'LOS ANGELES CITY',
        'LA COUNTY',
        'CONSOLIDATED FIRE PROTECTION', 
        'LACOUNTY', 
        'LAPD', 
        'LOS ANGELES POLICE',
               'LOS ANGELES POLICE DEPT AIR SUPPORT DIVISION',
       'COUNTY OF LOS ANGELES',
       'COUNTY OF LA',
              'CITY OF LOS ANGELES',
              'LOS ANGELES FIRE',
              'LOS ANGELES FIRE DEPARTMENT',
              'LOS ANGELES CITY']

In [29]:
la_cops_choppers = src[(src['name'].str.contains('|'.join(lapd_county))) \
                       & (src['state'] == 'CA') \
              & (src['type_aircraft'] == '6')]

In [30]:
la_safety_choppers_slim = la_cops_choppers\
    [[ 'n_number', 'name', 'mfr','model','street','city','year_mfr','no_seats' ]]

In [31]:
len(la_cops_choppers)

55

In [32]:
la_cops_choppers.to_csv('output/la_safety_choppers_slim.csv', index=False)

In [33]:
la_cops_choppers.n_number.unique().tolist()

['N306FD',
 'N664PD',
 'N3202Q',
 'N601CC',
 'N120LA',
 'N73985',
 'N213PF',
 'N221LA',
 'N223LA',
 'N267LA',
 'N472LA',
 'N703WP',
 'N704WP',
 'N110LA',
 'N14LA ',
 'N21844',
 'N228LA',
 'N229LA',
 'N232LA',
 'N233LA',
 'N661PD',
 'N662PD',
 'N665PD',
 'N913WB',
 'N961SD',
 'N963SD',
 'N966SD',
 'N301FD',
 'N304FD',
 'N303FD',
 'N305FD',
 'N950LA',
 'N951LA',
 'N953LA',
 'N954LA',
 'N955LA',
 'N956LA',
 'N957LA',
 'N958LA',
 'N959LA',
 'N960LA',
 'N961LA',
 'N962LA',
 'N302FD',
 'N668PD',
 'N17LA ',
 'N18LA ',
 'N15LA ',
 'N160LA',
 'N190LA',
 'N821LA',
 'N822LA',
 'N950SG',
 'N951LB',
 'N952JH']

---

### Bobcat fire aircraft

In [34]:
bobcat_src = pd.read_csv('input/aircraft.csv')

In [35]:
bobcat_src.n_number = bobcat_src.n_number.str.strip()

In [36]:
bobcat = bobcat_src.merge(master, on='n_number')

In [37]:
bobcat_merge = bobcat.merge(reference, left_on='mfr_mdl_code', right_on='code')

In [38]:
bobcat_slim = bobcat_merge[[ 'n_number', 'name', 'street', 'city', 'type_aircraft', 'mfr', 'model', 'no_seats']]

In [39]:
bobcat_slim.loc[(bobcat_slim['type_aircraft'] == '5'),'type']='Fixed wing multi engine'
bobcat_slim.loc[(bobcat_slim['type_aircraft'] == '6'),'type']='Rotorcraft'

/Users/mhustiles/.virtualenvs/notebooks-hnoN6uG0/lib/python3.7/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/mhustiles/.virtualenvs/notebooks-hnoN6uG0/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [40]:
bobcat_slim.drop(['type_aircraft'], axis=1)

,n_number,name,street,city,mfr,model,no_seats,type
0,N409DF,USDA FOREST SERVICE FEPP,5500 PRICE AVE,MCCLELLAN,NORTH AMERICAN,OV-10A,2,Fixed wing multi engine
1,N415DF,USDA FOREST SERVICE FEPP,5500 PRICE AVE,MCCLELLAN,NORTH AMERICAN,OV-10A,2,Fixed wing multi engine
2,N463DF,STATE OF CALIFORNIA - CAL FIRE,5500 PRICE AVE,MCCLELLAN,RAYTHEON AIRCRAFT COMPANY,A200CT,15,Fixed wing multi engine
3,N110LA,CONSOLIDATED FIRE PROTECTION,12605 OSBORNE ST,PACOIMA,BELL,412EP,15,Rotorcraft
4,N436DF,USDA FOREST SERVICE - FEPP,5500 PRICE AVE,MCCLELLAN,MARSH AVIATION,S-2F3AT,5,Fixed wing multi engine
5,N437DF,USDA FOREST SERVICE FEPP,5500 PRICE AVE,MCCLELLAN,MARSH AVIATION,S-2F3AT,5,Fixed wing multi engine
6,N137CG,DELAWARE TRUST CO TRUSTEE,251 LITTLE FALLS DR,WILMINGTON,BOEING,737-3H4,149,Fixed wing multi engine
7,N261CG,COULSON AVIATION (USA) INC,610 SW ALDER ST STE 910,PORTLAND,SIKORSKY,S-61N,28,Rotorcraft
8,N839AC,AERO-FLITE INC,8520 W ELECTRIC AVE,SPOKANE,BAE SYSTEMS PLC,AVRO 146-RJ85A,110,Fixed wing multi engine
9,N295EA,CORSAIR TWO LLC,2050 NE 25TH AVE,HILLSBORO,MCDONNELL DOUGLAS,DC-9-87,172,Fixed wing multi engine


In [41]:
bobcat_slim.to_csv('output/aircraft.csv', index=False)